In [58]:
# initial imports
import pandas as pd
from path import Path

In [59]:
# Load data
file_path = Path("../Resources/loans_data.csv")
loans_df = pd.read_csv(file_path)
loans_df.head()
#loans_df.sample(n=20)

amount  term      month  age             education  gender  bad
0    1000    30       June   45  High School or Below    male    0
1    1000    30       July   50              Bachelor  female    0
2    1000    30     August   33              Bachelor  female    0
3    1000    15  September   27               college    male    0
4    1000    30    October   28               college  female    0

In [60]:
# Months dictionary
months_num = {
    "January": 1,
    "February": 2,
    "March": 3,
    "April": 4,
    "May": 5,
    "June": 6,
    "July": 7,
    "August": 8,
    "September": 9,
    "October": 10,
    "November": 11,
    "December": 12,
}

In [61]:
#transform the text columns to numerical
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#transform the gender column
loans_df["gender"] = le.fit_transform(loans_df["gender"])
#transform the education column
loans_df["education"] = le.fit_transform(loans_df["education"])
#transform the month column using data dictionary: months_num
loans_df["month_num"] = loans_df["month"].apply(lambda x: months_num[x])
loans_df = loans_df.drop(["month"], axis=1)


In [62]:
loans_df.sample(n=10)

amount  term  age  education  gender  bad  month_num
328    1000    30   24          1       0    1          4
307    1000    30   40          1       1    1          4
88      800    15   43          0       1    0          5
330    1000    15   31          3       1    1         12
482     800    15   47          3       1    1          9
283    1000    30   30          1       1    0          9
122    1000    15   20          3       1    0          6
55     1000    15   25          3       1    0          7
92      800    15   38          1       1    0          3
282    1000    30   30          3       1    0         12

# OR THIS OPTION

In [63]:
alt_df = loans_df = pd.read_csv(file_path)
# using pandas to encode gender and education
alt_df = pd.get_dummies(alt_df, columns= ["education", "gender"])
alt_df["month"] = alt_df["month"].apply(lambda x: months_num[x])
#alt_df.drop(["month"])
alt_df.head()

amount  term  month  age  bad  education_Bachelor  \
0    1000    30      6   45    0                   0   
1    1000    30      7   50    0                   1   
2    1000    30      8   33    0                   1   
3    1000    15      9   27    0                   0   
4    1000    30     10   28    0                   0   

   education_High School or Below  education_Master or Above  \
0                               1                          0   
1                               0                          0   
2                               0                          0   
3                               0                          0   
4                               0                          0   

   education_college  gender_female  gender_male  
0                  0              0            1  
1                  0              1            0  
2                  0              1            0  
3                  1              0            1  
4                  1              1            0

# scale the data

In [64]:
# we'll first import the StandardScaler module and create an instance of it as data_scaler
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()

In [65]:
# scale the data
scaled_df = data_scaler.fit_transform(alt_df)
scaled_df

array([[ 0.49337687,  0.89789115, -0.16890147, ..., -0.88640526,
        -0.42665337,  0.42665337],
       [ 0.49337687,  0.89789115,  0.12951102, ..., -0.88640526,
         2.34382305, -2.34382305],
       [ 0.49337687,  0.89789115,  0.42792352, ..., -0.88640526,
         2.34382305, -2.34382305],
       ...,
       [-1.24386563, -0.97897162, -0.16890147, ...,  1.12815215,
        -0.42665337,  0.42665337],
       [ 0.49337687,  0.89789115, -1.06413896, ...,  1.12815215,
         2.34382305, -2.34382305],
       [ 0.49337687,  0.89789115, -1.06413896, ..., -0.88640526,
        -0.42665337,  0.42665337]])

In [66]:
scaled_df.shape

(500, 11)

In [67]:
#verify that the columns have been standardized/scaled
import numpy as np
print(np.mean(scaled_df[:,0]))
print(np.std(scaled_df[:,0]))

-3.552713678800501e-16
0.9999999999999999


In [68]:
# check that all of the columns have been scaled
int=0
while int < 11:
    print(f'column {int}')
    print(f'mean of col {int}: {np.mean(scaled_df[:,int])}')
    print(f'standard deviation of col {int}: {np.std(scaled_df[:,int])}')
    int+=1

column 0
mean of col 0: -3.552713678800501e-16
standard deviation of col 0: 0.9999999999999999
column 1
mean of col 1: -1.7763568394002506e-16
standard deviation of col 1: 1.0
column 2
mean of col 2: 6.838973831690964e-17
standard deviation of col 2: 1.0
column 3
mean of col 3: 6.039613253960852e-17
standard deviation of col 3: 1.0
column 4
mean of col 4: -5.684341886080802e-17
standard deviation of col 4: 0.9999999999999999
column 5
mean of col 5: -4.263256414560601e-17
standard deviation of col 5: 1.0
column 6
mean of col 6: 1.2168044349891717e-16
standard deviation of col 6: 0.9999999999999999
column 7
mean of col 7: -1.0658141036401503e-17
standard deviation of col 7: 1.0000000000000004
column 8
mean of col 8: 2.842170943040401e-17
standard deviation of col 8: 1.0
column 9
mean of col 9: -2.842170943040401e-17
standard deviation of col 9: 1.0
column 10
mean of col 10: 3.907985046680551e-17
standard deviation of col 10: 1.0


# fitting data to model

In [69]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [70]:
# define the features set (all the columns except the one we want to predict (bad))
#create a copy of the unscaled df
X = alt_df.copy()
#drop the target column
X = X.drop("bad", axis=1)
# show table
X.head()

amount  term  month  age  education_Bachelor  \
0    1000    30      6   45                   0   
1    1000    30      7   50                   1   
2    1000    30      8   33                   1   
3    1000    15      9   27                   0   
4    1000    30     10   28                   0   

   education_High School or Below  education_Master or Above  \
0                               1                          0   
1                               0                          0   
2                               0                          0   
3                               0                          0   
4                               0                          0   

   education_college  gender_female  gender_male  
0                  0              0            1  
1                  0              1            0  
2                  0              1            0  
3                  1              0            1  
4                  1              1            0

In [71]:
# define the target set
y = alt_df["bad"].values
# show the first 5 rows in y
y[:5]

array([0, 0, 0, 0, 0])

In [72]:
# split the data into test and train sets default is 75% train, 25% test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state =78)

In [73]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(375, 10)
(125, 10)
(375,)
(125,)


In [74]:
# Splitting into Train and Test sets into an 80/20 split.
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, random_state=78, train_size=0.80)


In [75]:
# To scale our data, we'll use the StandardScaler as before and fit the instance, scaler, with the training data and then scale the features with the transform() method:
# create a standardscalar instance
scaler = StandardScaler()
# fit the standard scaler with the TRAINING data
X_scaler = scaler.fit(X_train)

# scaling the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [76]:
# Determine the shape of our training and testing sets.
print(X_train2.shape)
print(X_test2.shape)
print(y_train2.shape)
print(y_test2.shape)

(400, 10)
(100, 10)
(400,)
(100,)


In [77]:
# check that all of the columns have been scaled
int=0
while int < 10:
    print(f'column {int}')
    print(f'mean of col {int}: {np.mean(X_train_scaled[:,int])}')
    print(f'standard deviation of col {int}: {np.std(X_train_scaled[:,int])}')
    print(f'mean of col {int}: {np.mean(X_test_scaled[:,int])}')
    print(f'standard deviation of col {int}: {np.std(X_test_scaled[:,int])}')
    int+=1

column 0
mean of col 0: 3.931669804539221e-16
standard deviation of col 0: 1.0
mean of col 0: 0.08040483006321758
standard deviation of col 0: 0.8450480061575104
column 1
mean of col 1: -4.618527782440651e-17
standard deviation of col 1: 1.0
mean of col 1: 0.12297037557744318
standard deviation of col 1: 0.9564007345405279
column 2
mean of col 2: -1.0894988614988202e-16
standard deviation of col 2: 1.0
mean of col 2: -0.24508261791538438
standard deviation of col 2: 0.9902565532613558
column 3
mean of col 3: -1.1131836193574902e-16
standard deviation of col 3: 0.9999999999999999
mean of col 3: -0.046669098854475274
standard deviation of col 3: 1.0135597524939095
column 4
mean of col 4: 1.4210854715202004e-17
standard deviation of col 4: 1.0
mean of col 4: 0.10470260847804559
standard deviation of col 4: 1.1072483741768515
column 5
mean of col 5: 0.0
standard deviation of col 5: 1.0
mean of col 5: 0.1689329399014945
standard deviation of col 5: 1.0209540692403776
column 6
mean of col 6:

# fitting decision tree model

In [78]:
# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier()
# Fitting the model.
model = model.fit(X_train_scaled, y_train)

In [79]:
# Making predictions using the testing data.
predictions = model.predict(X_test_scaled)

In [80]:
# look at preds
predictions


array([1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0])

In [81]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

Predicted 0  Predicted 1
Actual 0           52           32
Actual 1           22           19

In [82]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
print(acc_score)

0.568


In [83]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


Predicted 0  Predicted 1
Actual 0           52           32
Actual 1           22           19

Accuracy Score : 0.568
Classification Report
              precision    recall  f1-score   support

           0       0.70      0.62      0.66        84
           1       0.37      0.46      0.41        41

    accuracy                           0.57       125
   macro avg       0.54      0.54      0.54       125
weighted avg       0.59      0.57      0.58       125



## Precision: 
#### Precision is the measure of how reliable a positive classification is. From our results, the precision for the good loan applications can be determined by the ratio TP/(TP + FP), which is 50/(50 + 22) = 0.69. The precision for the bad loan applications can be determined as follows: 19/(19 + 34) = 0.358. A low precision is indicative of a large number of false positives—of the 53 loan applications we predicted to be bad applications, 34 were actually good loan applications.
## Recall: 
#### Recall is the ability of the classifier to find all the positive samples. It can be determined by the ratio: TP/(TP + FN), or 50/(50 + 34) = 0.595 for the good loans and 19/(19 + 22) = 0.463 for the bad loans. A low recall is indicative of a large number of false negatives.
## F1 score: 
#### F1 score is a weighted average of the true positive rate (recall) and precision, where the best score is 1.0 and the worst is 0.0.
## Support: 
#### Support is the number of actual occurrences of the class in the specified dataset. For our results, there are 84 actual occurrences for the good loans and 41 actual occurrences for bad loans.

# Random forest

In [84]:
from sklearn.ensemble import RandomForestClassifier

In [85]:
# first we define the features set
# Define the features set.
X = alt_df.copy()
X = X.drop("bad", axis=1)
X.head()

amount  term  month  age  education_Bachelor  \
0    1000    30      6   45                   0   
1    1000    30      7   50                   1   
2    1000    30      8   33                   1   
3    1000    15      9   27                   0   
4    1000    30     10   28                   0   

   education_High School or Below  education_Master or Above  \
0                               1                          0   
1                               0                          0   
2                               0                          0   
3                               0                          0   
4                               0                          0   

   education_college  gender_female  gender_male  
0                  0              0            1  
1                  0              1            0  
2                  0              1            0  
3                  1              0            1  
4                  1              1            0

In [86]:
# Define the target set.
y = alt_df["bad"].ravel()
y[:5]

array([0, 0, 0, 0, 0])

In [87]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [88]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [89]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=500, random_state=78) 

In [90]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [91]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)
predictions

array([0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1])

In [92]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

Predicted 0  Predicted 1
Actual 0           50           34
Actual 1           25           16

In [93]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [94]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


Predicted 0  Predicted 1
Actual 0           50           34
Actual 1           25           16

Accuracy Score : 0.528
Classification Report
              precision    recall  f1-score   support

           0       0.67      0.60      0.63        84
           1       0.32      0.39      0.35        41

    accuracy                           0.53       125
   macro avg       0.49      0.49      0.49       125
weighted avg       0.55      0.53      0.54       125



In [95]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.05605314, 0.08174556, 0.32422431, 0.42849715, 0.02069517,
       0.02259031, 0.00269276, 0.02465162, 0.01924365, 0.01960632])

In [96]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.42849715251905834, 'age'),
 (0.3242243146649606, 'month'),
 (0.0817455625673348, 'term'),
 (0.05605313793135903, 'amount'),
 (0.024651620072561973, 'education_college'),
 (0.022590308645694574, 'education_High School or Below'),
 (0.020695166815155628, 'education_Bachelor'),
 (0.019606324846519876, 'gender_male'),
 (0.019243654066394316, 'gender_female'),
 (0.0026927578709609414, 'education_Master or Above')]

In [97]:
# first we define the features set
# Define the features set.
X = alt_df.copy()
X = X[["age", "month", "term", "amount", "education_college", "education_High School or Below"]]
X.head()
# Define the target set.
y = alt_df["bad"].ravel()
y[:5]
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=500, random_state=78) 
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)
predictions
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

Confusion Matrix


Predicted 0  Predicted 1
Actual 0           50           34
Actual 1           25           16

Accuracy Score : 0.528
Classification Report
              precision    recall  f1-score   support

           0       0.67      0.60      0.63        84
           1       0.32      0.39      0.35        41

    accuracy                           0.53       125
   macro avg       0.49      0.49      0.49       125
weighted avg       0.55      0.53      0.54       125



[(0.42849715251905834, 'amount'),
 (0.3242243146649606, 'term'),
 (0.0817455625673348, 'month'),
 (0.05605313793135903, 'age'),
 (0.022590308645694574, 'education_High School or Below'),
 (0.020695166815155628, 'education_college')]

In [99]:
# first we define the features set
# Define the features set.
X = alt_df.copy()
X = X[["age", "month", "term", "amount"]]

# Define the target set.
y = alt_df["bad"].ravel()

# Creating a StandardScaler instance.
scaler = StandardScaler()

# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=500, random_state=78) 

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

Confusion Matrix


Predicted 0  Predicted 1
Actual 0           50           34
Actual 1           25           16

Accuracy Score : 0.528
Classification Report
              precision    recall  f1-score   support

           0       0.67      0.60      0.63        84
           1       0.32      0.39      0.35        41

    accuracy                           0.53       125
   macro avg       0.49      0.49      0.49       125
weighted avg       0.55      0.53      0.54       125



[(0.42849715251905834, 'amount'),
 (0.3242243146649606, 'term'),
 (0.0817455625673348, 'month'),
 (0.05605313793135903, 'age')]